<a href="https://colab.research.google.com/github/tanayparsai20/Playstore-EDA-Analysis/blob/main/Prediction_of_Temperature_Using_Time_Series_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
global_temp=pd.read_csv('/content/drive/MyDrive/New folder/GlobalTemperatures.csv')

FileNotFoundError: ignored

In [ ]:
global_temp.head()

In [ ]:
global_temp.columns.values

In [ ]:
global_temp.drop(columns=['LandMaxTemperature',
       'LandMaxTemperatureUncertainty', 'LandMinTemperature',
       'LandMinTemperatureUncertainty', 'LandAndOceanAverageTemperature',
       'LandAndOceanAverageTemperatureUncertainty'],inplace=True)

In [ ]:
global_temp.describe()

In [ ]:
global_temp.isnull().sum()

In [ ]:
global_temp.dropna(inplace=True)

In [ ]:
global_temp.shape

In [ ]:
global_temp['dt']=pd.to_datetime(global_temp['dt'])

In [ ]:
global_temp.dtypes

# 2. Does Global Warming Exist

In [ ]:
global_temp['years']=global_temp['dt'].dt.year

In [ ]:
global_temp.head()

In [ ]:
data=global_temp.groupby('years').agg({'LandAverageTemperature':'mean','LandAverageTemperatureUncertainty':'mean'}).reset_index()

In [ ]:
data

Maximum Uncertainty will be Avg temperature  +   deviation(Uncertainty)

whereas minimum Uncertainty will be Avg temp -   deviation(Uncertainty)

and this deviation is exactly your Uncertainty...

In [ ]:
data['Uncertainty_top']=data['LandAverageTemperature'] + data['LandAverageTemperatureUncertainty']
data['Uncertainty_bottom']=data['LandAverageTemperature'] - data['LandAverageTemperatureUncertainty']

In [ ]:
data.head()

In [ ]:
data.plot(x='years',y=['LandAverageTemperature', 'LandAverageTemperatureUncertainty',
       'Uncertainty_top', 'Uncertainty_bottom'],figsize=(16,10))

# 3. Analyse average temperature in each season

In [ ]:
## extract 'month' from 'dt' feature.
global_temp['month']=global_temp['dt'].dt.month

In [ ]:
global_temp.head()


In [ ]:
## lets define a function to extract season from 'month'

def get_season(month):

    if month>=3 and month<=5:
        return 'spring'

    elif month>=6 and month<=8:
        return 'summer'

    elif month>=9 and month<=11:
        return 'autumn'

    else:
        return 'winter'

In [ ]:
global_temp['season']=global_temp['month'].apply(get_season)

In [ ]:
global_temp.columns

In [ ]:
years=global_temp['years'].unique()

In [ ]:
global_temp['season'].unique()

In [ ]:
## lets define 4 blank lists to store all the "season temp"  of "various years".

spring_temps=[]
summer_temps=[]
autumn_temps=[]
winter_temps=[]

for year in years:
    current_yr=global_temp[global_temp['years']==year]

    spring_temps.append(current_yr[current_yr['season']=='spring']['LandAverageTemperature'].mean())
    summer_temps.append(current_yr[current_yr['season']=='summer']['LandAverageTemperature'].mean())
    autumn_temps.append(current_yr[current_yr['season']=='autumn']['LandAverageTemperature'].mean())
    winter_temps.append(current_yr[current_yr['season']=='winter']['LandAverageTemperature'].mean())

In [ ]:
### lets make a blank dataframe ..

season=pd.DataFrame()

In [ ]:
## assigning features in dataframe..

season['years']=years
season['spring_temps']=spring_temps
season['summer_temps']=summer_temps
season['autumn_temps']=autumn_temps
season['winter_temps']=winter_temps

In [ ]:
season

In [ ]:
season.columns

In [ ]:
season[['spring_temps', 'summer_temps', 'autumn_temps',
       'winter_temps']]

In [ ]:
plt.figure(figsize=(16,10))
plt.plot(season['years'],season[['spring_temps', 'summer_temps', 'autumn_temps',
       'winter_temps']])

plt.legend(['spring_temps', 'summer_temps', 'autumn_temps',
       'winter_temps'])

In [ ]:
Is it getting warmer? Yes, it is.

# 4. How to prepare data for time series Modelling


In [ ]:
### lets read 'GlobalLandTemperaturesByCity.csv'

cities=pd.read_csv('/content/drive/MyDrive/New folder/GlobalLandTemperaturesByCity.csv')

In [ ]:
cities.head()


In [ ]:
cities.shape

In [ ]:
## finding unique countries in 'country' feature..

cities['Country'].unique()

In [ ]:
## creating separate datafrme for 'India' only..

India=cities[cities['Country']=='India']

In [ ]:
India.columns

In [ ]:
## unique cities in 'India'..

India['City'].unique()

In [ ]:
## considering data of ['Delhi','Bombay','Madras''Visakhapatnam','Jammu','Hyderabad','Imphal'] in my India dataframe..

India_cities=['Delhi','Bombay','Madras''Visakhapatnam','Jammu','Hyderabad','Imphal']

In [ ]:
data2=India[India['City'].isin(India_cities)]

In [ ]:
data2.shape

In [ ]:
data2.head()

In [ ]:
data2=data2[['dt','AverageTemperature']]



In [ ]:
data2.columns=['Date','Temp']

In [ ]:
data2

In [ ]:
data2.dtypes

In [ ]:
## converting data-type of 'Date' feature into date-time ..

data2['Date']=pd.to_datetime(data2['Date'])

In [ ]:
data2.dtypes

In [ ]:
## total missing values in each of the feature..

data2.isnull().sum()

In [ ]:
## dropping the missing values..

data2.dropna(inplace=True)

In [ ]:
data2.head()

In [ ]:
data2.shape

In [ ]:
data2.set_index('Date',inplace=True)

In [ ]:
data2.head()

# 5.How to find weather data has seasonality factor or not

In [ ]:
we will try to convert our seasonal data into stationary bcz Time SEries algos works phenomenal with stationary data..

lets understand what is seasoanlity ,stationary & couple of Time series Terms..

seasonality ->> When mean & variance is not constant throughout the data ,
                ie data has a trend here..

stationary data ->>  so data that has no trend ie that has constant
                   mean & std dev throughout data..

if not stationary then we have to make it stationary using various manipulations..


Seasonality is opposite of stationary ,ie
    A time series with a clear seasonal component is referred to as non-stationary...

    ie the goal is we have to convert seasonal data into stationary data..



    a)Visualisation approach

(seasonality is all about suppose in each yr in christmas sales goes up) and then down ,
   ie this is a cycle that repeats over time, such as monthly or yearly.

   There are many types of seasonality;
   for example:

Time of Day.
Daily.
Weekly.
Monthly.
Yearly.
Once seasonality is identified, it can be removed

The model of seasonality can be removed from the time series.
This process is called Seasonal Adjustment, or Deseasonalizing.
A time series where the seasonal component has been removed is called seasonal stationary.
A time series with a clear seasonal component is referred to as non-stationary.

In [ ]:
###  i'm going to create a pivot table to plot the monthly temperatures through the years
## so I need 'year' & 'month' feature..

data2['year']=data2.index.year

In [ ]:
data2['month']=data2.index.month

In [ ]:
data2.head()

In [ ]:
## creating a pivot table :

pivot=data2.pivot_table(values='Temp',index='month',columns='year')

In [ ]:
pivot

In [ ]:
## if we want to plot with respect to time or index of dataframe ,  we can use pivot.plot() function of pandas ..


pivot.plot(figsize=(20,8))
plt.legend().remove()
plt.xlabel('Months')
plt.ylabel('Temperatures')

In [ ]:
# looking this graph,we can say this data is seasonal
#     The series clearly has some seasonality, the higher temperatures
#     are around b/w April & May and the lower are between December & Feb
#     so thats basically a observation that u can drawn and represent to ur client..

# 6. How to find weather data is stationary or not


ways to find

a.. using data visualisation (lineplot to find out
                           whats a trend of data)

b.. Evaluating the descriptive statistics ( ie compute
             mean & variance of various chunks of data )

 c.. using statistical Tests (AD-Fuller , KPSS Test )

In [ ]:
### checking whether data is stationary or not..

data2['Temp'].plot(figsize=(20,12))

In [ ]:
'''
It seems that data is seasonal in nature.. The spread of the data indicates that there is a significant variation in the data.
To flatten the growing variance, we need to transform the data.

A recurring pattern with a defined and predictable regularity dependent on the time of year, week, or day
is referred to as seasonality.



In [ ]:
### Testing For Stationarity using Augmented Dickey-Fuller test..

from statsmodels.tsa.stattools import adfuller

In [ ]:
adfuller(data2['Temp'])

In [ ]:
adfuller gives 5 values ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used','critical values']

most imp is p-value
p-value concept is almost similar to Hypothesis testing,
Hypothesis Testing have 2 Hypothesis(Alternate & Null Hypothesis)

Null hypo says Data is not Stationary
whereas Alternate hypo says data is stationary

lags are those which are our previous Data


In [ ]:
#if Pvalue<0.05 ,we reject Null hypo ie alternate hypo is
# true,ie data is stationery

In [ ]:
here p value is greater than 0.05 hence we accept null hypothesis that data is not stationary

# 7. How to make your data stationary

In [ ]:
data2['Temp'].head(15)

In [ ]:
data2['first_temp_diff']=data2['Temp'] - data2['Temp'].shift(12)

In [ ]:
'''
why taken shift(12) , bcz basically year has 12 month cycle , ie in order to remove seasonal component..

In previous diagram of pivot table , we have figured out that we have yearly seasonality.. ,
so lets use shift(12) to remove seasonality

'''

In [ ]:
data2.head(15)

In [ ]:
### Testing For Stationarity using Augmented Dickey-Fuller test..

from statsmodels.tsa.stattools import adfuller

In [ ]:
## Again perform Aug.dickey fuller test on df['first_diff_temp']

adfuller(data2['first_temp_diff'].dropna())

In [ ]:
''
now our p-value is less than 0.05 which basically says we are rejecting null hypo and accepting alternate hypo
ie data is stationery

Note : if p-value is almost 0, then we can say , we have a wonderful stationary graph

'''

In [ ]:
data2['first_temp_diff'].plot(figsize=(20,6))

#8. Lets Build Time series Model

In [ ]:
# Lets Build a basic moving Avg model

In [ ]:
df=data2[['first_temp_diff']]

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.head(10)

In [ ]:
### Smoothening your series using moving average
## Rolling is just like a window that you are going to consider or I can say its a window that we have considered for Moving average,
## lets say 5 is the window size or 5 is the business period


df['Predictions']=df['first_temp_diff'].rolling(window=5).mean()

In [ ]:
df.head()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.head()

In [ ]:
df.columns=['actual_temp','forecast_temp']

In [ ]:
df.head()

In [ ]:
### check accuracy of your model using RMSE..

from sklearn.metrics import mean_squared_error

In [ ]:
np.sqrt(mean_squared_error(df['actual_temp'],df['forecast_temp']))

## why np.sqrt, bcz mean_squared_error gives square of error and then by calling np.sqrt we get actual error

In [ ]:
plt.plot(df["actual_temp"])
plt.plot(df["forecast_temp"])

# 9. Lets Build ARIMA Model

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.graphics.api import qqplot

In [ ]:
df.head()

In [ ]:
plot_acf(df['actual_temp'])
plt.show()

In [ ]:
plot_pacf(df['actual_temp'])
plt.show()

In [ ]:
df.head(3)

In [ ]:
df=df[['actual_temp']]

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
training_data=df[0:10000]
testing_data=df[10000:]

### create test data that is basically unseen to our model or which is basically is used for evalaution

In [ ]:
from statsmodels.tsa.arima_model import ARIMA


In [ ]:
### arima= ARIMA(training_data,order=(p,d,q))

arima=ARIMA(training_data,order=(2,1,2))

In [ ]:
model=arima.fit()

In [ ]:
model.summary()

In [ ]:
## doing predictions..

pred=model.forecast(steps=len(testing_data))[0]

In [ ]:
pred

In [ ]:
len(pred)

In [ ]:
len(testing_data)

In [ ]:
training_data.plot(legend=True, label='Train', figsize=(20,6))
testing_data.plot(legend=True, label='Train')
pred.plot(legend=True, label='predictionARIMA')

In [ ]:
 arima=ARIMA(testing_data,order=(2,1,2))
model=arima.fit()
pred=model.forecast(steps=len(testing_data))[0]
np.sqrt(mean_squared_error(testing_data,pred))
model.summary()


In [ ]:
Error=df["actual_temp"]-pred